In [3]:
#######################################################
#### Set this value to the directory of period-suite ##
#######################################################
pathToSuite="/Users/sertoez/git-projects/period-suite/"
#######################################################

import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
defaultPrecision=1e-100;
# Do not forget to increase the precision of initial conditions if going beyond e-100
#
#
#
precision=[]; #if empty we will use the default precision for all systems 
paths=[]; #unless overridden we will convert this to a list of straight paths

t0=time.time();
load(pathToSuite+"current.sage")
t1=time.time();
print "Loaded file in", t1-t0, "seconds";
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "\nIntegrating system number", j;
    transitionMatricies = []
    for i in [1..noOfdeqs]:
        tt0=time.time()
        tm=deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true)
        print "\tODE number", i, "took", time.time()-tt0, "seconds to integrate"
        transitionMatricies=transitionMatricies+[tm]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    print "The largest error:"
    print max([max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies])
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(2000), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:noOfdeqs]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print Matrix(periods).str()



# Write the periods to file

def row2string(row):
    strRow=str(row[0])
    for i in [1..(len(row)-1)]:
        strRow=strRow+", "+str(row[i])
    return "["+strRow+"]"
    
outputFile = open(pathToSuite+"lastPeriods",'w')

# Change default precision here or in Magma if you want to load them
outputFile.write("SetDefaultRealFieldPrecision(100);\n")
outputFile.write("CC<I>:=ComplexField(100);\n")
outputFile.write("M:=Matrix(["+row2string(periods[0]))
for i in [1..(periods.nrows()-1)]:
    outputFile.write(",\n"+row2string(periods[i]))
outputFile.write("]);")
outputFile.close()


Loaded file in 0.898227930069 seconds
Integrating systems 1 through 7

Integrating system number 1
	ODE number 1 took 1.15494704247 seconds to integrate
	ODE number 2 took 1.44801092148 seconds to integrate
	ODE number 3 took 1.09148097038 seconds to integrate
	ODE number 4 took 1.38431382179 seconds to integrate
	ODE number 5 took 0.925363063812 seconds to integrate
	ODE number 6 took 0.750185966492 seconds to integrate
	ODE number 7 took 1.02360606194 seconds to integrate
	ODE number 8 took 1.58483600616 seconds to integrate
	ODE number 9 took 1.03154087067 seconds to integrate
	ODE number 10 took 1.1032781601 seconds to integrate
	ODE number 11 took 1.58775997162 seconds to integrate
	ODE number 12 took 1.20287895203 seconds to integrate
	ODE number 13 took 0.936731100082 seconds to integrate
	ODE number 14 took 1.19037103653 seconds to integrate
	ODE number 15 took 1.55103683472 seconds to integrate
	ODE number 16 took 1.07322096825 seconds to integrate
	ODE number 17 took 0.986001

In [4]:
#### Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 80
##
## verify that you found honest relations by checking suspected relations using more digits
##
verify= 90
##
##

load(pathToSuite+"IntersectionMatrix.sage")

periodVector=periods
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])

proj = M.transpose()
Proj = (10^scale*proj).apply_map(lambda x : 10^3*x.round())
lattice = block_matrix([[Proj, matrix.identity(Proj.dimensions()[0])]])
reducedLattice = lattice.LLL()

Proj2 = (10^verify*proj).apply_map(lambda x : x.round())
testRelations = [v[2:23] * Proj2 for v in reducedLattice.rows()]

norms1 = [reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]
norms2 = [testRelations[i].norm().n()+reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]

norms=norms2

sortedNorms=sorted(norms)
consecutiveRatios = [sortedNorms[i]/sortedNorms[i-1] for i in [1..20]]

j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if sortedNorms[0] > 50 :
    rank = 1

print "\nPicard rank appears to be", rank, "\n"
print "Check to see if this makes sense:\n"
print "First row records the norm of the relation."
print "Second row shows if the relation continues to hold with more digits.\n"

print "01","||", "---hyperplane section---"
j=0
while j <= 7 :
    print "0" + str(j+2), "||", norms1[j], testRelations[j]
    j=j+1
while j <= 20 :
    print j+2, "||", norms1[j], testRelations[j]
    j=j+1



Picard rank appears to be 4 

Check to see if this makes sense:

First row records the norm of the relation.
Second row shows if the relation continues to hold with more digits.

01 || ---hyperplane section---
02 || 2.82842712474619 (0, 1)
03 || 3.74165738677394 (0, -1)
04 || 6.16441400296898 (0, 2)
05 || 9.59442767022046e8 (-2597333275268064968, 2190192449195093911)
06 || 1.04868529301691e9 (4402761980693874302, -3495624492659623920)
07 || 1.09931466125027e9 (-6187761419734487669, -113033708602415877)
08 || 1.06159230438496e9 (-4567927213612015403, -152879584525169863)
09 || 1.12733429070984e9 (-1937501755576933429, -1982240346014639798)
10 || 1.04661450361762e9 (1478208984777425779, -1547850852407399534)
11 || 1.18920169528685e9 (3619032524310805510, -2299098623023098956)
12 || 1.18477039516717e9 (1541351756486500592, -1803519097498773554)
13 || 1.11469815538377e9 (-5623215720788485484, 4542912618381913903)
14 || 1.28598028191057e9 (-2969024847127869090, 5299143745265464556)
15 || 1

In [11]:
reducedLattice.row(17)[2:23]

(1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0)